# Turbulence example

In this notebook we show how to perform a simulation using a soundspeed field based on a Gaussian turbulence spectrum.

In [ ]:
import numpy as np
from pstd import PSTD, PML, Medium, Position2D, PointSource
from pstd import PSTD
from acoustics import Signal
from turbulence import Field2D, Gaussian2DTemp
#import seaborn as sns
%matplotlib inline

## Configuration

The following are the parameters for our simulation

In [ ]:
x = 50.0
y = 40.0
z = 0.0

c_0 = 343.2
maximum_frequency_target = 200.0

## Turbulent field

In [ ]:
f_max = 500.0

f_margin = 1.0

# Amount of modes
max_mode_order = 100
# Maximum wavenumber
k_max = 10.0

wavenumber_resolution = k_max / max_mode_order

# We don't need it for the calculations but we do need it to create an instance.
spatial_resolution = c_0 / (2.0 * f_max * f_margin)

spectrum = Gaussian2DTemp(
    max_mode_order=max_mode_order,
    wavenumber_resolution=wavenumber_resolution,
    mu_0=3e-2,
    a=1.1,
#     a=0.001,
)

field = Field2D(
    x=x,
    y=y,
    z=y,
    spatial_resolution=spatial_resolution,
    spectrum=spectrum,
)

mu = field.randomize().generate().mu

print("Mu shape: {}".format(mu.shape))
c = ( mu + 1.0 ) * c_0

field.plot()

## Create model

We now create a model. Waves propagate in a medium which we define first.

In [ ]:
medium = Medium(soundspeed=c, density=1.296)

The model is only finite and to prevent aliasing we need a Perfectly Matched Layer.

In [ ]:
pml = PML((1000.0, 1000.0), depth=10.0)

Now we create the actual model.

In [ ]:
model = PSTD(
    maximum_frequency=maximum_frequency_target,
    pml=pml,
    medium=medium,
    cfl=PSTD.maximum_cfl(medium.soundspeed)/2.,
    size=[x, y],
    spacing = spatial_resolution,
)

In this example our source excites a pulse. 

In [ ]:
source_position = Position2D(x*2.0/5.0, y/2.0)

source = model.add_object('source', 'PointSource', position=source_position, 
                         excitation='pulse', quantity='pressure', amplitude=0.1)

We also add a receiver on the other side of the domain

In [ ]:
receiver_position = Position2D(x*3.0/5.0, y/2.0)

receiver = model.add_object('receiver', 'Receiver', position=receiver_position, quantity='pressure')

## Check model

To get a quick overview of all parameters, for example to check them, we can print one

In [ ]:
print(model.overview())

To check whether the geometry is as we want it to be, we can simply draw it.

In [ ]:
_ = model.plot_scene()

## Running the simulation

Now that we've defined and checked our model we can run it.

With `model.run()` you can specify the amount of time steps or amount of seconds it should run.

In [ ]:
model.run(seconds=0.01)

Let's see how the sound pressure field looks like now.

In [ ]:
_ = model.plot_field()

It might happen that you realize that you actually need to calculate a bit further. This can easily be done, since the state is remembered. Simply use `model.run()` again and the simulation continues.

In [ ]:
model.run(seconds=0.03)

With more steps we now see the effect of the turbulence.

In [ ]:
_ = model.plot_field()

We can also check the signal recorded by the receiver, which in this case is the impulse response. The method `receiver.recording()` returns an instance of `acoustics.Signal`.

In [ ]:
_ = receiver.recording().plot()

If however, you want to restart the simulation you can do so with `model.restart()`.

In [ ]:
model.restart()